# 定義


## 表示式上の文字を構造体Edgeで定義し、展開図上の点をVertexで定義する

In [94]:
class Edge:
    def __init__(self, n, inv):
      """
      辺の定義

      n: 表示式の文字であり、任意の文字数を表現できるように整数値で表現する
      inv: 向きが反転しているかどうかを示すフラグ
      """

      self.n = n
      self.inv = inv

class Vertex:
  def __init__(self, edge_id1, is_src1, edge_id2, is_src2):
    """
    頂点の定義

    edge_id1: 辺1
    is_src1: 辺1の始点かどうかを示すフラグ
    edge_id2: 辺2
    is_src2: 辺2の始点かどうかを示すフラグ
    """

    self.edge_id1 = edge_id1
    self.is_src1 = is_src1
    self.edge_id2 = edge_id2
    self.is_src2 = is_src2

  def other(self, edge_id, is_src):
    """
    辺1との対応があれば辺2の関係を返す
    辺2との対応があれば辺1の関係を返す
    それ以外はNoneを返す
    """

    if edge_id == self.edge_id1 and is_src == self.is_src1:
      return self.edge_id2, self.is_src2
    elif edge_id == self.edge_id2 and is_src == self.is_src2:
      return self.edge_id1, self.is_src1
    else:
      None


## メソッドの用意

In [90]:
# 向き付け可能性の判定
def is_orientable(Surface):
  """
  向き付け可能性を判定する

  Surface: 閉局面の表示式
  """

  for i, e in enumerate(Surface):
    for f in Surface[i+1:]:
      # 1つでも同符号の点が見つかった場合
      if e.n == f.n and e.inv == f.inv:
        return False # 向き付け不可能
  # すべて異符号の場合
  return True # 向き付け可能

In [91]:
# 頂点を作り出す
def make_vertices(Surface):
  """
  展開図上の頂点を作り出す

  Surface: 閉局面の表示式
  """

  vertices = []
  for i, e in enumerate(Surface):
    vertex = Vertex(
        e.n,
        not e.inv,
        Surface[i-1].n,
        Surface[i-1].inv
        )
    vertices.append(vertex)
  return vertices

In [104]:
# インデックスに対応する頂点を全てリストに格納
def make_group(idx, Surface, item_to_process=None):
  """
  idx番目の頂点に対応する頂点のインデックスリストと
  item_to_processを返す

  idx: 頂点のインデックス
  Surface: 閉局面の表示式
  item_to_process: 処理すべき頂点のインデックスリスト
  """

  vertices = make_vertices(Surface)
  edge_id, is_src = vertices[idx].edge_id1, vertices[idx].is_src1
  group = [idx]
  item_to_process.remove(idx) if item_to_process else None

  break_flag = True
  while break_flag:
    break_flag = False
    for i in item_to_process:
      other = vertices[i].other(edge_id, is_src)
      if other:
        edge_id, is_src = other
        idx = i
        break_flag = True
        break

    if break_flag:
      group.append(idx)
      item_to_process.remove(idx) if item_to_process else None

  return group, item_to_process


In [105]:
# 展開図上の点を分類
def classify_vertices(Surface):
  """
  展開図上の点を分類

  Surface: 閉局面の表示式
  """

  vertices = make_vertices(Surface)
  item_to_process = list(range(len(Surface)))
  groups = []

  while item_to_process:
    idx = item_to_process[0]
    group, item_to_process = make_group(idx, Surface, item_to_process)
    groups.append(group)
  return groups

In [106]:
# 表示式から閉局面を判定する
def classify_surface(Surface):
  """
  表示式から閉局面を判定して出力

  Surface: 閉局面の表示式
  """

  orientable  = is_orientable(Surface)
  groups = classify_vertices(Surface)

  # V, E, Fからchi(S)を計算
  V = len(groups)
  E = len(Surface) // 2
  F = 1
  chi = V - E + F

  if orientable:
    if chi == 2:
      print("S2")
    else:
      n = (2 - chi) // 2
      print(f"T2({n})")
  else:
    n = 2 - chi
    print(f"P2({n})")

# 実践

## 辺と表示式の定義

In [107]:
a = Edge(1, False)
b = Edge(2, False)
c = Edge(3, False)
d = Edge(4, False)
e = Edge(5, False)

a_inv = Edge(1, True)
b_inv = Edge(2, True)
c_inv = Edge(3, True)
d_inv = Edge(4, True)
e_inv = Edge(5, True)

S = [a, b, c_inv, a_inv, d_inv, e, b_inv, d, e_inv, c]

## 向き付け可能性
向き付け可能：True

向き付け不可能：False

In [108]:
is_orientable(S)

True

## 同じ点のグループを見つける
リストの数が実質的な点の個数(=V)を表す
リストの中身の数値は多角形の各頂点に番号を振ったもの

In [109]:
classify_vertices(S)

[[0, 4, 8, 6, 2], [1, 7, 5, 9, 3]]

## 表示式から閉局面を分類

In [110]:
classify_surface(S)

T2(2)


# 実行のみ

In [112]:
S = [a, a, c_inv, b, c_inv, b_inv]
classify_surface(S)

P2(3)


In [113]:
S = [a, b, c, a_inv, d_inv, e, b_inv, d, e_inv, c]
classify_surface(S)

P2(5)


In [114]:
S = [a, a_inv]
classify_surface(S)

S2


In [116]:
S = [a, a, b, c, b_inv, c_inv]
classify_surface(S)

P2(3)


In [117]:
S = [a_inv, d, e, b_inv, e, c_inv, a, d_inv, c_inv, b_inv]
classify_surface(S)

P2(4)
